# Mutually exclusive groups

TODO

### VRP with alternative clients

We continue with a VRP variant that involves selecting a client from a set of alternatives.
This situation arises in applications where deliveries can be dropped off at different locations (e.g., pickup points) or when clients request multiple, disjunctive time windows (e.g., morning or evening delivery).
PyVRP's client groups provide a general way to model such situations.
Each client group represents a set of alternative clients, and for each group, exactly one client must be selected.

Let's demonstrate this with our rolling example, where we introduce four different client groups.

In [ ]:
# This defines each location's group membership.
GROUPS = [None, None, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3]

In [ ]:
# m = Model()

# for idx in range(2):
#     depot = m.add_depot(x=COORDS[idx][0], y=COORDS[idx][1])
#     m.add_vehicle_type(
#         2,
#         start_depot=depot,
#         end_depot=depot,
#         tw_early=TIME_WINDOWS[idx][0],
#         tw_late=TIME_WINDOWS[idx][1],
#     )

# # Create the client groups. Clients are assigned to their group below.
# groups = [m.add_client_group() for _ in range(4)]

# for idx in range(2, len(COORDS)):
#     m.add_client(
#         x=COORDS[idx][0],
#         y=COORDS[idx][1],
#         tw_early=TIME_WINDOWS[idx][0],
#         tw_late=TIME_WINDOWS[idx][1],
#         prize=PRIZES[idx],
#         required=False,
#         group=groups[GROUPS[idx]],  # assign group
#     )

# for frm_idx, frm in enumerate(m.locations):
#     for to_idx, to in enumerate(m.locations):
#         distance = abs(frm.x - to.x) + abs(frm.y - to.y)  # Manhattan
#         duration = DURATION_MATRIX[frm_idx][to_idx]
#         m.add_edge(frm, to, distance=distance, duration=duration)

In [ ]:
# res = m.solve(stop=MaxRuntime(1), display=False)  # one second
# print(res)

In [ ]:
# _, ax = plt.subplots(figsize=(8, 8))
# plot_solution(res.best, m.data(), plot_clients=True, ax=ax)

# # Plot each client's group label.
# for client in m.clients:
#     ax.text(client.x + 10, client.y + 10, f"{client.group}")